In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

2025-06-08 08:30:16.478323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749371416.744522      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749371416.823287      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/celeba-dataset/list_attr_celeba.csv")
df_age = pd.read_csv("/kaggle/input/celeba-dataset/list_eval_partition.csv")

In [3]:
df.replace({-1: 0}, inplace=True)

In [4]:
print(df.columns.tolist())

['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']


In [5]:
df['long_hair'] = ((df['Wavy_Hair'] == 1) | (df['Straight_Hair'] == 1)).astype(int)

In [6]:
df['image_id'] = df['image_id'].astype(str)

In [7]:
df['image_id'] = df['image_id'].astype(str)

In [8]:
df = df.sample(n=40000, random_state=42).reset_index(drop=True)

In [9]:
np.random.seed(42)
df['age'] = np.random.randint(15, 50, df.shape[0])

In [10]:
def map_gender(row):
    if 20 <= row['age'] <= 30:
        return 'female' if row['long_hair'] == 1 else 'male'
    else:
        return 'female' if row['Male'] == 0 else 'male'

In [11]:
df['gender_label'] = df.apply(map_gender, axis=1)

In [12]:
train_df, val_df = train_test_split(df[['image_id', 'gender_label']], test_size=0.2, random_state=42)
image_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_dataframe(train_df, directory=image_dir, x_col='image_id', y_col='gender_label', target_size=(128, 128), class_mode='binary', batch_size=32)
val_gen = datagen.flow_from_dataframe(val_df, directory=image_dir, x_col='image_id', y_col='gender_label', target_size=(128, 128), class_mode='binary',batch_size=32)

Found 32000 validated image filenames belonging to 2 classes.
Found 8000 validated image filenames belonging to 2 classes.


In [13]:
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(64, (3,3), activation='relu'))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(128, (3,3), activation='relu'))
cnn.add(MaxPooling2D(2,2))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add((Dropout(0.5)))
cnn.add(Dense(32, activation='relu'))
cnn.add((Dropout(0.5)))
cnn.add((Dense(1, activation='sigmoid')))

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
cnn.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stop])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-08 08:33:18.931567: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 622s 617ms/step - accuracy: 0.6628 - loss: 0.6068 - val_accuracy: 0.8306 - val_loss: 0.4254
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 566s 566ms/step - accuracy: 0.8275 - loss: 0.4526 - val_accuracy: 0.8395 - val_loss: 0.4096
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 604s 604ms/step - accuracy: 0.8413 - loss: 0.4274 - val_accuracy: 0.8469 - val_loss: 0.3987
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 565s 565ms/step - accuracy: 0.8436 - loss: 0.4222 - val_accuracy: 0.8469 - val_loss: 0.3997
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 592s 592ms/step - accuracy: 0.8497 - loss: 0.4000 - val_accuracy: 0.8453 - val_loss: 0.4030
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 566s 566ms/step - accuracy: 0.8554 - loss: 0.3970 - val_accuracy: 0.8499 - val_loss: 0.3983
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 550s 550ms/step - accuracy: 0.8576 - loss: 0.3858 - val_accuracy: 0.8400 - val_loss: 0.4119
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 551s 551ms/step - ac

In [21]:
cnn.save('long_hair_prediction_model.h5')